<h1><center> Lab 5: Network (Data) Science and Assignment 2 Support </center></h1>

Assignment 2 will combine supervised learning on networks with an essay to reflect on the ethical implications of data science. In this notebook, you can practice with network analysis (community detection and link prediction).

You can also use parts of this notebook as a starting point for your assignment: the final part of this notebook can help you to import network data, perform feature engineering and output your predictions.

<h2> Community detection </h2>

The first task we will consider is community detection. This is one example of unsupervised learning with network data. Community detection methods are helpful to detect communities (also known as clusters of modules) based on network structure, such as strongly connected groups of nodes: groups of friends in social, academic sub-communities in co-authorship networks, groups of related web-pages, groups of functionally similar units in metabolic networks.

Here you'll find some portions of code that you can use to perform Community Detection analysis (discussed in [Lecture 7](https://canvas.uva.nl/courses/45979/files/11537488?module_item_id=2187507)). We're going to use the example of the Zachary's Karate Club introduced in Lecture 8. You can find more details about this example [here](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Generate a network (NetworkX already provides a method to extract the Zachary's Karate Club data)
G = nx.karate_club_graph()

# Lets displace nodes following a Directed Force algorithm (remember Lecture 4, on Data Viz, and Lab 2 Aux)
pos = nx.spring_layout(G)

# Draw the original graph
nx.draw(G, pos=pos)

We're now going to play with an implementation of Community Detection offered by NetworkX, which will follow the same principles of Modularity Maximization lectured in [Lecture 7](https://canvas.uva.nl/courses/45979/files/11537488?module_item_id=2187507).

You can find extra details about function <code>greedy_modularity_communities</code> [here](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html).

In [ ]:
from networkx.algorithms.community import greedy_modularity_communities

#the function will return a partition of nodes
c = list(greedy_modularity_communities(G, resolution=0.8))

print(c)

#we can inspecte the first element of the partition (e.g., nodes on community 1)
sorted(c[0])

**Q1: How many communities are identified? What is the impact of parameter resolution (now set to 0.8) on this value?**

(check the  greedy_modularity_communities documentation pointed above)

**Q2: Can you plot the network with nodes colores based on communities?**

<h2> Link Prediction - Auxiliary functions Assignment 2 </h2>

The next task we consider is link prediction. Link prediction will be at the core of your [Assignment 2](https://canvas.uva.nl/courses/45979/assignments/537063).

Predicting links on a network is fundamental to predict future connections in social networks or discover hidden connections in incomplete network data. You can find more info about link prediction on [Wikipedia](https://en.wikipedia.org/wiki/Link_prediction).


There are many approaches to link prediction, from simpler ones (based on heuristics and network similarity measures, also called topology-based approaches) to more complex ones (based on graph embeddings and deep learning).

The supervised learning methods you learned so far can also be used to perform link prediction. In your Assignment you should use supervised learning, in particular the techniques discussed in the course, to tackle the problem of link prediction. In the example below, you will try to predict links in the Karate Club network.

In [ ]:
# Importing and exporting network data
import pandas as pd

# We will start by exporting and importing the Karate Club network
# This example can be useful for you to import the network data used in Assignment 2

edges = []
for edg in nx.edges(G):
    edges.append(edg)

dfEdges = pd.DataFrame(edges)
dfEdges.to_csv("karateclub.edgelist", index=False, header=False)

G1 = nx.read_edgelist("karateclub.edgelist", data=False, nodetype = int, delimiter=',') # import
nx.draw(G1, pos=pos)

Above, we are creating a network by importing a list of edges in the exact same format as edges_train.edgelist for Assignment 2.

In the code below, we will create a training set by removing nEdgesRemove random edges from the network above. The removed links will be visualised in red.

In [ ]:
# We will now create a training set by removing nEdgesRemove random edges from the network above
# 10 edges will be removed
nEdgesRemove = 10

# Sample random edges
allEdges = np.array(G1.edges)
np.random.seed(seed=42)
selectEdges = np.random.choice(np.arange(allEdges.shape[0]), nEdgesRemove)
edgesToRemove = allEdges[selectEdges]

# Create new graph (H) without selected edges
H = G1.copy()
H.remove_edges_from(edgesToRemove)

# Visualize the links that were removed (in red)
# Use the same colors as defined above to color communities
for i in range(len(c)):
    nx.draw_networkx_nodes(G, pos, nodelist=sorted(c[i]))

nx.draw_networkx_edges(G, pos, edge_color='red')
nx.draw_networkx_edges(H, pos, edge_color='black')

#Compute number of nodes in the network
N = len(list(H.nodes))
print("The network has ", N, " nodes")

#Compute number of edges in the network
E = len(list(H.edges))
print("The network has ", E, " edges")

After importing the network the next task is to define the features that seem important to predict the occurence of links.

This is the step where feature engineering can take place. We are using [preferential attachment](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_prediction.preferential_attachment.html#networkx.algorithms.link_prediction.preferential_attachment) as one feature to infer links. This might not be the most interesting feature to predict links in this social network. Other possible features can be constructed by metrics you can learn about in the NetworkX documentation ([here](https://networkx.org/documentation/stable/reference/algorithms/link_prediction.html)).

**Q2: Which other features can you use? Adapt function *getFeature* to consider other feastures associated with each edge.**

In [ ]:
# Use Preferential Attachment between nodes as a feature
pa = np.zeros((N,N))

preds = nx.preferential_attachment(H, [(i,j) for i in range(N) for j in range(N)])
for u, v, p in preds:
    pa[u,v] = p

# Generate features from edge endpoints
# Input: getFeature(graph, node_i, node_j)
def getFeature(G, i, j): 
    return ###

The next step is to create a training set. You can do this by using all visible edges in network H (positive examples) and some non-existing edges in network H (negative examples).

In [ ]:
# Let us now create a training set where X will correspond to features for each possible edge and Y the predciton
X = []
Y = []

# Add positive examples where edge exist
for (i, j) in H.edges:
    X.append(getFeature(H, i, j))
    Y.append(1)

# Add negative examples where edge does not exist
# Reflect: how many negative examples should you include?
for kk in range(200):
        i = np.random.randint(N)
        j = np.random.randint(N)

        while H.has_edge(i,j) or i == j:
            i = np.random.randint(N)
            j = np.random.randint(N)

        X.append(getFeature(H, i, j))
        Y.append(0)

**Q3: In the previous code we are adding 200 negative examples (non-existing edges) to our traning set. Is that a reasonable choice? How many positive examples are we considering?**

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.8, random_state = 42)

In [ ]:
# Tune model (Logistic Regression)

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

clf = LogisticRegression(random_state=0, C=2.0)
clf.fit(X_train, y_train)

scores = cross_val_score(clf, X_test, y_test, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

**Q4: The previous code trains a Logistic regression with a specific hyper-parameter. Which other learning models could you use, out of the ones discussed in the course? How could you select the best hyper-parameter?**

Given a list of edges, we should be able to predict its existence. Below we provide an example of importing an input file and returning, for edge, the prediction of the trained model.

**Q5: Please complete the missing text to generate the predictions from your model given the input data.**

In [ ]:
# Return Solution
inpTest = pd.read_csv('aux_assignment2_testInput.csv', sep=',', index_col="ID")
inpTest = np.array(inpTest)
inp = []

for i in inpTest:
    inp.append(getFeature(H, i[0], i[1]))

predictionsLR = ## Line to complete

In [ ]:
# Output solution
pd.DataFrame(predictionsLR).to_csv('aux_assignment2_predictions.csv', index=True, header=["prediction"], index_label="ID")

In [ ]:
# Evaluate solution

# Read predictions
y_pred = pd.read_csv('aux_assignment2_predictions.csv', sep=',', index_col="ID")
y_pred = np.array(y_pred)

# Read solutions file
y = pd.read_csv('aux_assignment2_solution.csv', sep=',', index_col="ID")
y = np.array(y)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print("Accuracy (test): %0.3f%% " % (accuracy * 100))

**Q6: Reflect on the accuracy value you obtained. What does that mean? Knowing that in the final test set there are 50% of positive examples and 50% of negative examples, what would be the accuracy of a random classifier? What would be the accuracy of a naive classifier always predicting that a link does not exist?**